# Computing samples from the predictive, each time updated

In [1]:
def print_PT_predictive(ax, all_alpha, all_B, sample):

    final_intervals = all_B[-1]
    _, sampled_distribution = sample_from_polya_tree(all_alpha)

    # Plot della densità empirica
    kde = sns.kdeplot(sample, ax=ax, color='red', label='Densità empirica', linewidth=2)
    
    x_kde, y_kde = kde.get_lines()[0].get_data()  # Get x and y values from the Line2D object
    interpolator = interp1d(x_kde, y_kde, kind='linear', bounds_error=False, fill_value="extrapolate")
    x_values = sample
    y_values = interpolator(x_values)

    ax.plot(x_values, y_values, 'ro')

    # Plot della predittiva
    sampled_distribution = compute_predictive(all_alpha, all_B)
    widths = np.diff(final_intervals, axis=1).flatten()
    area = np.sum(sampled_distribution * widths)
    normalized_distribution = sampled_distribution / area
    f = interp1d(x_points, normalized_distribution, kind='linear', fill_value="extrapolate")
    x_interp = np.linspace(final_intervals[0, 0], final_intervals[-1, 1], 100)
    y_interp = f(x_interp)
    ax.plot(x_interp, y_interp, alpha=1, color = "black", label = fr"Predittiva $y_{{{len(sample)+1}}}$")


    # Aggiungi etichette e titolo
    ax.plot()
    ax.legend()
    ax.grid()
    
    return sampled_distribution

In [ ]:
final_sample_size = 12
c = 1
depth = 10

fig, ax = plt.subplots(4,4, figsize = (18,18))

sample = np.random.uniform(0, 1, 2)
# sample = np.random.normal(0.5, 1/10, 2)       

# I want to compute the predictive for a new data and sample from it, then repeat
for i in range(4):
    for j in range(4):
        all_alpha, all_B = polya_tree_posterior(data = sample, c = c, depth = depth)
        new_predictive = print_PT_predictive(ax[i,j], all_alpha, all_B, sample)
        ax[i,j].set_title(f'n={len(sample)}')

        new_obs = sample_from_categorical_distribution(all_B[-1], new_predictive, 1)   #sampling from the predictive

        sample = np.append(sample, new_obs)
        # sample = np.append(sample, new_obs)